In [44]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movielens-20m-dataset/rating.csv
/kaggle/input/movielens-20m-dataset/link.csv
/kaggle/input/movielens-20m-dataset/genome_tags.csv
/kaggle/input/movielens-20m-dataset/genome_scores.csv
/kaggle/input/movielens-20m-dataset/tag.csv
/kaggle/input/movielens-20m-dataset/movie.csv


In [45]:
mov_df = pd.read_csv('/kaggle/input/movielens-20m-dataset/movie.csv')
rat_df = pd.read_csv('/kaggle/input/movielens-20m-dataset/rating.csv')
gen_tag_df = pd.read_csv('/kaggle/input/movielens-20m-dataset/genome_tags.csv')
gen_sco_df = pd.read_csv('/kaggle/input/movielens-20m-dataset/genome_scores.csv')
tag_df = pd.read_csv('/kaggle/input/movielens-20m-dataset/tag.csv')
link_df = pd.read_csv('/kaggle/input/movielens-20m-dataset/link.csv')


In [46]:
mov_rat = pd.merge(mov_df,rat_df)
mov_rat

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3,4.0,1999-12-11 13:36:47
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6,5.0,1997-03-13 17:50:52
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8,4.0,1996-06-05 13:37:51
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10,4.0,1999-11-25 02:44:47
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11,4.5,2009-01-02 01:13:41
...,...,...,...,...,...,...
20000258,131254,Kein Bund für's Leben (2007),Comedy,79570,4.0,2015-03-30 19:32:59
20000259,131256,"Feuer, Eis & Dosenbier (2002)",Comedy,79570,4.0,2015-03-30 19:48:08
20000260,131258,The Pirates (2014),Adventure,28906,2.5,2015-03-30 19:56:32
20000261,131260,Rentun Ruusu (2001),(no genres listed),65409,3.0,2015-03-30 19:57:46


In [47]:
gen_sco_tag = pd.merge(gen_tag_df,gen_sco_df)
gen_sco_tag.head()

,tagId,tag,movieId,relevance
0,1,007,1,0.02500
1,1,007,2,0.03975
2,1,007,3,0.04350
3,1,007,4,0.03725
4,1,007,5,0.04200


In [48]:
tag_df.head()
tag_df = tag_df.drop(['timestamp'],axis=1)

In [49]:
gen_sco_tag = pd.merge(gen_sco_tag,tag_df)

In [50]:
gen_sco_tag

,tagId,tag,movieId,relevance,userId
0,1,007,10,0.99975,8527
1,1,007,10,0.99975,11074
2,1,007,10,0.99975,14260
3,1,007,10,0.99975,77058
4,1,007,10,0.99975,81725
...,...,...,...,...,...
206148,1128,zombies,103249,0.98225,108442
206149,1128,zombies,103249,0.98225,122523
206150,1128,zombies,103249,0.98225,138209
206151,1128,zombies,111663,0.81825,88738


this looks better with all tags sorted together, I will sort it into one big datframe later

In [51]:
link_df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


Guess i wont be needing this

So now we have the neccesary data and thus we will combine it and do some preprocessing

In [52]:
df = pd.merge(mov_rat,gen_sco_tag)
df.head()

,movieId,title,genres,userId,rating,timestamp,tagId,tag,relevance
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1741,4.0,2002-11-27 18:31:23,244,computer animation,0.99850
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3596,1.0,2010-11-21 04:00:52,64,animation,0.98575
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3596,1.0,2010-11-21 04:00:52,374,family,0.84850
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6233,4.5,2014-01-23 02:14:34,1114,witty,0.77675
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10616,3.5,2010-06-24 05:25:56,29,adventure,0.89200


In [53]:
df = df.drop(['timestamp','tagId','tag','relevance'],axis=1)
df

,movieId,title,genres,userId,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1741,4.0
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3596,1.0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3596,1.0
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6233,4.5
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10616,3.5
...,...,...,...,...,...
183141,128488,Wild Card (2015),Crime|Drama|Thriller,11084,5.0
183142,128488,Wild Card (2015),Crime|Drama|Thriller,11084,5.0
183143,128604,Knight of Cups (2015),Drama,78564,0.5
183144,128604,Knight of Cups (2015),Drama,78564,0.5


In [54]:
df['rating']=df['rating'].astype(int)
df

,movieId,title,genres,userId,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1741,4
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3596,1
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3596,1
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6233,4
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10616,3
...,...,...,...,...,...
183141,128488,Wild Card (2015),Crime|Drama|Thriller,11084,5
183142,128488,Wild Card (2015),Crime|Drama|Thriller,11084,5
183143,128604,Knight of Cups (2015),Drama,78564,0
183144,128604,Knight of Cups (2015),Drama,78564,0


In [55]:
number_rating = mov_rat.groupby('title')['rating'].count().reset_index()

In [56]:
number_rating.rename(columns={'rating':'number of rating'},inplace=True)
number_rating.head()

,title,number of rating
0,#chicagoGirl: The Social Network Takes on a Di...,3
1,$ (Dollars) (1971),24
2,$5 a Day (2008),39
3,$9.99 (2008),55
4,$ellebrity (Sellebrity) (2012),2


In [57]:
df = pd.merge(df,number_rating)
df

,movieId,title,genres,userId,rating,number of rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1741,4,49695
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3596,1,49695
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3596,1,49695
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6233,4,49695
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10616,3,49695
...,...,...,...,...,...,...
183141,128488,Wild Card (2015),Crime|Drama|Thriller,11084,5,20
183142,128488,Wild Card (2015),Crime|Drama|Thriller,11084,5,20
183143,128604,Knight of Cups (2015),Drama,78564,0,2
183144,128604,Knight of Cups (2015),Drama,78564,0,2


In [58]:
df.shape

(183146, 6)

In [59]:
df=df[df['number of rating']>=50]

In [60]:
df.drop_duplicates(['title','userId'],inplace=True)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [61]:
df.shape

(73949, 6)

In [62]:
df.head()

,movieId,title,genres,userId,rating,number of rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1741,4,49695
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3596,1,49695
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6233,4,49695
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10616,3,49695
12,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11081,2,49695


In [63]:
df.drop(columns=['number of rating'],inplace=True)
df.head()

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,movieId,title,genres,userId,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1741,4
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3596,1
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6233,4
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10616,3
12,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11081,2


In [64]:
mov_piv=df.pivot_table(columns='userId',index='title',values='rating')

In [65]:
mov_piv.shape

(6843, 5070)

In [66]:
mov_piv.fillna(0,inplace=True)

In [67]:
mov_piv

userId,65,96,121,129,133,190,279,316,318,320,...,138233,138254,138280,138287,138301,138345,138406,138414,138436,138446
title,,,,,,,,,,,,,,,,,,,,,
$9.99 (2008),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"'burbs, The (1989)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
(500) Days of Summer (2009),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
*batteries not included (1987),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...And Justice for All (1979),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
[REC]² (2009),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
eXistenZ (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
xXx (2002),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [68]:
from scipy.sparse import csr_matrix
mov_spar=csr_matrix(mov_piv)

In [69]:
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(n_neighbors=10,algorithm='brute',metric='cosine')

In [70]:
knn.fit(mov_spar)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=10)

In [71]:
df.drop(columns=['genres','userId','rating'],inplace=True)


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [72]:
df.drop_duplicates(inplace=True)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [73]:
df.to_csv('RecoSys.csv',index=False)

In [74]:
df_f =df.copy()
fin=[]
for i in df_f['title']:
    fin.append(i.split(' (')[0])
df_f['title']=fin

In [75]:
def reco(mov_name):
    movie_id=df_f[df_f['title']=='Toy Story'].drop_duplicates('title')['movieId'].values[0]
    distances,suggestions=knn.kneighbors(mov_piv.iloc[movie_id,:].values.reshape(1,-1))
    
    
    
    for i in range(len(suggestions)):
        return (mov_piv.index[suggestions[i]])

In [76]:
res=reco('xXx(2002)')

In [ ]:
for i in res:
    print(i)